## Notes
---
### SCD Type 2 Triggers (new row):
- `supplier_name`
- `supplier_category`
- `city`
- `delivery_frequency`
- `email`

> Create new row, assign new `supplier_key`, mark `is_active` as `True`, mark old row's `is_active` as `False`

### SCD Type 1 Triggers (overwrite):
- `notes`
- `is_active`

> Just overwrite these bro
---

# Extract

In [ ]:
# load data/day1/supplier_details.csv and suppliers.csv into pandas
import pandas as pd

details = pd.read_csv("../data/day1/supplier_details.csv")
suppliers = pd.read_csv("../data/day1/suppliers.csv")

   SupplierID                     Email DeliveryFrequency  \
0         101  contact@beanroasters.com            Weekly   
1         102       info@sweettreats.ca             Daily   
2         103        sales@maplemilk.ca      Twice-Weekly   
3         104       support@cuptech.com           Monthly   
4         105        info@freshbrew.net            Weekly   

                        Notes  
0    Premium arabica supplier  
1     Specializes in pastries  
2         Local dairy partner  
3  Provides espresso machines  
4        Cold chain logistics  


# Clean

In [11]:
def clean(df_supp, df_det):
    # strip whitespace from strings
    df_supp = df_supp.map(lambda x: x.strip() if isinstance (x, str) else x)
    df_det = df_det.map(lambda x: x.strip() if isinstance (x, str) else x)

    # rename columns for consistenncy (i am not going to bother with regex)
    df_supp = df_supp.rename(columns={
        'SupplierID': 'supplier_id',
        'SupplierName': 'supplier_name',
        'SupplierCategory': 'supplier_category',
        'City': 'city',
        'IsActive': "is_active"
    })

    df_det = df_det.rename(columns={
        'SupplierID':'supplier_id',
        'Email': 'email',
        'DeliveryFrequency': 'delivery_frequency',
        'Notes': 'notes'
    })

    # lowercase all emails
    df_det["email"] = df_det["email"].str.lower()

    # suppliers.dtypes already shows 'is_active' as bool

    return df_supp, df_det

suppliers, details = clean(suppliers, details)
print("suppliers:",suppliers.head(3))
print("\n" + "-"*90)
print("details:",details.head(3))
print(f"{"|"*35} [Data Types for Reference] {"|"*35}")
print(f"{suppliers.dtypes} \n\n\n {details.dtypes}")


suppliers:    supplier_id       supplier_name supplier_category         city  is_active
0          101    BeanRoasters Co.      Coffee Beans      Toronto       True
1          102  SweetTreats Bakery            Bakery  Mississauga       True
2          103     MapleMilk Dairy      Milk & Dairy     Brampton       True

------------------------------------------------------------------------------------------
details:    supplier_id                     email delivery_frequency  \
0          101  contact@beanroasters.com             Weekly   
1          102       info@sweettreats.ca              Daily   
2          103        sales@maplemilk.ca       Twice-Weekly   

                      notes  
0  Premium arabica supplier  
1   Specializes in pastries  
2       Local dairy partner  
||||||||||||||||||||||||||||||||||| [Data Types for Reference] |||||||||||||||||||||||||||||||||||
supplier_id           int64
supplier_name        object
supplier_category    object
city                 obj

# Staging